# FinBERT BaseLine


## Modules

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from pathlib import Path
import shutil

import os
import logging
import sys
sys.path.append('..')
from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from google.colab import drive
drive.mount('/content/drive')


#!cp /content/drive/finBERT-master/finBERT-master.finbert.finbert.py .
!cp -r /content/drive/MyDrive/finBERT-master/finBERT-master/finbert .
from finbert.finbert import *
import finbert.utils as tools


%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-2-1c6b0791d3d3>:31: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [3]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

## Prepare the model

In [4]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
cl_data_path = project_dir/'data'/'sentiment_data'

###  Configuring training parameters

In [5]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path)
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

In [6]:
finbert = FinBert(config)
finbert.base_model = 'bert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [7]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

## Fine-tune the model

In [8]:
# Get the training examples
train_data = finbert.get_data('train')

In [9]:
model = finbert.create_the_model()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Training

In [10]:
trained_model = finbert.train(train_examples = train_data, model = model)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/109 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Validation losses: [0.36050240580852216]
No best model found


Epoch:  25%|██▌       | 1/4 [00:16<00:50, 16.79s/it]

...

Validation losses: [0.36050240580852216, 0.394982805618873, 0.3875008305678001]


Iteration:   0%|          | 0/109 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch: 100%|██████████| 4/4 [01:30<00:00, 22.71s/it]

Validation losses: [0.36050240580852216, 0.394982805618873, 0.3875008305678001, 0.3869855168920297]


## Test the model

`bert.evaluate` outputs the DataFrame, where true labels and logit values for each example is given

In [11]:
test_data = finbert.get_data('test')

In [12]:
results = finbert.evaluate(examples=test_data, model=trained_model)

Testing:   0%|          | 0/31 [00:00<?, ?it/s]

### Prepare the classification report

In [13]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [14]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [15]:
report(results,cols=['labels','prediction','predictions'])

Loss:0.41
Accuracy:0.83

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.84      0.79       267
           1       0.73      0.90      0.81       128
           2       0.91      0.82      0.86       575

    accuracy                           0.83       970
   macro avg       0.80      0.85      0.82       970
weighted avg       0.84      0.83      0.83       970



<ipython-input-13-c8c15b3723e1>:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))


### Get predictions

With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns:

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [16]:
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [17]:
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
result = predict(text,model)

In [20]:
blob = TextBlob(text)
result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
result

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,"Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China.","[0.0018319299, 0.98960215, 0.0085659465]",negative,-0.987770,0.051746
1,The news rapidly infected financial markets.,"[0.004011223, 0.9658445, 0.03014418]",negative,-0.961833,0.000000
2,Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened.,"[0.0017531227, 0.99091655, 0.00733037]",negative,-0.989163,0.500000
3,"The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground.","[0.012198636, 0.9804755, 0.0073259557]",negative,-0.968277,0.000000
4,Asian stockmarkets closed down on January 3rd and European ones opened lower.,"[0.0018882557, 0.99099207, 0.007119689]",negative,-0.989104,-0.051111
5,Yields on government bonds fell as investors fled to the traditional haven in a market storm.,"[0.002478914, 0.98728, 0.010241015]",negative,-0.984801,0.000000


In [21]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is -0.98.


Here is another example

In [22]:
text2 = "Shares in the spin-off of South African e-commerce group Naspers surged more than 25% \
in the first minutes of their market debut in Amsterdam on Wednesday. Bob van Dijk, CEO of \
Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the \
Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019. REUTERS/Piroschka van de Wouw \
Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a \
31% stake in Chinese tech titan Tencent. There is 'way more demand than is even available, so that’s \
good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg. 'It’s going to be an interesting \
hour of trade after opening this morning.' Euronext had given an indicative price of 58.70 euros \
per share for Prosus, implying a market value of 95.3 billion euros ($105 billion). The shares \
jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT."

In [23]:
result2 = predict(text2,model)
blob = TextBlob(text2)
result2['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]

In [24]:
result2

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,Shares in the spin-off of South African e-commerce group Naspers surged more than 25% in the first minutes of their market debut in Amsterdam on Wednesday.,"[0.9794534, 0.008575285, 0.011971285]",positive,0.970878,0.250000
1,"Bob van Dijk, CEO of Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019.","[0.012840245, 0.010072966, 0.97708684]",neutral,0.002767,0.000000
2,"REUTERS/Piroschka van de Wouw Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a 31% stake in Chinese tech titan Tencent.","[0.048673145, 0.003416729, 0.9479101]",neutral,0.045256,0.000000
3,"There is 'way more demand than is even available, so that’s good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg.","[0.97567016, 0.0065904884, 0.017739302]",positive,0.969080,0.533333
4,'It’s going to be an interesting hour of trade after opening this morning.',"[0.9220723, 0.0093412455, 0.06858637]",positive,0.912731,0.500000
5,"Euronext had given an indicative price of 58.70 euros per share for Prosus, implying a market value of 95.3 billion euros ($105 billion).","[0.0605724, 0.004702599, 0.934725]",neutral,0.055870,0.000000
6,The shares jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT.,"[0.18633243, 0.0065774657, 0.8070901]",neutral,0.179755,0.000000


In [25]:
print(f'Average sentiment is %.2f.' % (result2.sentiment_score.mean()))

Average sentiment is 0.45.


In [26]:
#!pip install tensorflow==2.14.0
#!pip install update urllib3

#!pip install textattack


from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
#model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")




In [27]:

import textattack
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(trained_model, tokenizer)


##Generate attack

In [55]:
datalist_test = []
for data in test_data:
  if(data.label=='positive'):
      l=1
  elif(data.label=='negative'):
      l=-1
  else:
      l=0
  datalist_test.append((data.text,l))
  #print(data.guid)
print(datalist_test)
dataset_test = textattack.datasets.Dataset(datalist_test)
datalist_train = []
for data in train_data:

  datalist_train.append((data.guid,data.text,data.label))
dataset_train = textattack.datasets.Dataset(datalist_train, input_columns=(["text"]))


print(len(dataset_test.input_columns))

[('The Bristol Port Company has sealed a one million pound contract with Cooper Specialised Handling to supply it with four 45-tonne , customised reach stackers from Konecranes ', 1), ('A paper mill in the central Maine town of Madison soon will have a new owner ', 0), ('ALEXANDRIA , Va. , Oct. 23 -- Hans-Otto Scheck of Espoo , Finland , has developed a method of identifying remote radio units in a communication system ', 0), ('Altona stated that the private company of Altona chairman Kevin Maloney , Tulla Resources , would take up its entitlement in full ', 0), ('Registration is required ', 0), ('Comparable operating profit for the quarter decreased from EUR510m while sales increased from EUR860m , as compared to the third quarter 2007 ', -1), ("`` Because we 're a pension insurance company , we 're required to diversify and not put too much in one asset class ", 0), ('The Hayward , Calif.-based target designs active , casual and dress footwear , as well as boots and sandals ', 0), ('

In [64]:
dataset = textattack.datasets.HuggingFaceDataset("financial_phrasebank",subset ="sentences_50agree")
# Attack 20 samples with CSV logging and checkpoint saved every 5 interval
attack = textattack.attack_recipes.TextFoolerJin2019.build(model_wrapper)
attack_args = textattack.AttackArgs(num_examples=4000, log_to_csv="log.csv", checkpoint_interval=100, checkpoint_dir="checkpoints", disable_stdout=True)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Loading datasets dataset financial_phrasebank, subset sentences_50agree, split train.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


  0%|          | 0/4000 [00:00<?, ?it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|          | 1/4000 [00:00<04:34, 14.58it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|          | 2/4000 [00:00<04:19, 15.41it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:   0%|          | 2/4000 [00:00<05:20, 12.47it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 3 / 3:   0%|          | 3/4000 [00:00<05:06, 13.03it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 3 / 3:   0%|          | 4/4000 [00:00<04:42, 14.12it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 4 / 4:   0%|          | 4/4000 [00:00<05:16, 12.61it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 5 / 5:   0%|          | 5/4000 [00:00<05:04, 13.10it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 5 / 5:   0%|          | 6/4000 [00:00<04:43, 14.09it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 6 / 6:   0%|          | 6/4000 [00:00<05:13, 12.72it/s]
[Succeeded / Failed / Skipped /


[Succeeded / Failed / Skipped / Total] 0 / 1 / 104 / 105:   3%|▎         | 105/4000 [00:19<12:03,  5.39it/s]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 104 / 105:   3%|▎         | 106/4000 [00:19<11:57,  5.43it/s]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 105 / 106:   3%|▎         | 106/4000 [00:19<11:58,  5.42it/s]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 106 / 107:   3%|▎         | 107/4000 [00:19<11:52,  5.46it/s]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 107 / 108:   3%|▎         | 108/4000 [00:19<11:47,  5.50it/s]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 107 / 108:   3%|▎         | 109/4000 [00:19<11:42,  5.54it/s]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 108 / 109:   3%|▎         | 109/4000 [00:19<11:42,  5.53it/s]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 109 / 110:   3%|▎         | 110/4000 [00:19<11:38,  5.57it/s]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 110 / 111:   3%|▎         | 111/4000 [00:19<11:34,  5.60it/s]
[Succeeded / Faile


[Succeeded / Failed / Skipped / Total] 2 / 1 / 201 / 204:   5%|▌         | 204/4000 [00:28<08:48,  7.18it/s]
[Succeeded / Failed / Skipped / Total] 2 / 1 / 201 / 204:   5%|▌         | 205/4000 [00:28<08:46,  7.21it/s]
[Succeeded / Failed / Skipped / Total] 2 / 1 / 202 / 205:   5%|▌         | 205/4000 [00:28<08:46,  7.20it/s]
[Succeeded / Failed / Skipped / Total] 2 / 1 / 203 / 206:   5%|▌         | 206/4000 [00:28<08:45,  7.23it/s]
[Succeeded / Failed / Skipped / Total] 2 / 1 / 204 / 207:   5%|▌         | 207/4000 [00:28<08:43,  7.25it/s]
[Succeeded / Failed / Skipped / Total] 2 / 1 / 204 / 207:   5%|▌         | 208/4000 [00:28<08:41,  7.27it/s]
[Succeeded / Failed / Skipped / Total] 2 / 1 / 205 / 208:   5%|▌         | 208/4000 [00:28<08:41,  7.27it/s]
[Succeeded / Failed / Skipped / Total] 2 / 1 / 206 / 209:   5%|▌         | 209/4000 [00:28<08:39,  7.29it/s]
[Succeeded / Failed / Skipped / Total] 2 / 1 / 207 / 210:   5%|▌         | 210/4000 [00:28<08:38,  7.32it/s]
[Succeeded / Faile

...


[Succeeded / Failed / Skipped / Total] 153 / 4 / 3544 / 3701:  93%|█████████▎| 3702/4000 [10:53<00:52,  5.67it/s]
[Succeeded / Failed / Skipped / Total] 153 / 4 / 3545 / 3702:  93%|█████████▎| 3702/4000 [10:53<00:52,  5.67it/s]
[Succeeded / Failed / Skipped / Total] 153 / 4 / 3546 / 3703:  93%|█████████▎| 3703/4000 [10:53<00:52,  5.67it/s]
[Succeeded / Failed / Skipped / Total] 154 / 4 / 3546 / 3704:  93%|█████████▎| 3704/4000 [10:55<00:52,  5.65it/s]
[Succeeded / Failed / Skipped / Total] 154 / 4 / 3546 / 3704:  93%|█████████▎| 3705/4000 [10:55<00:52,  5.65it/s]
[Succeeded / Failed / Skipped / Total] 154 / 4 / 3547 / 3705:  93%|█████████▎| 3705/4000 [10:55<00:52,  5.65it/s]
[Succeeded / Failed / Skipped / Total] 154 / 4 / 3548 / 3706:  93%|█████████▎| 3706/4000 [10:55<00:52,  5.65it/s]
[Succeeded / Failed / Skipped / Total] 154 / 4 / 3549 / 3707:  93%|█████████▎| 3707/4000 [10:55<00:51,  5.65it/s]
[Succeeded / Failed / Skipped / Total] 154 / 4 / 3549 / 3707:  93%|█████████▎| 3708/400


[Succeeded / Failed / Skipped / Total] 157 / 4 / 3640 / 3801:  95%|█████████▌| 3801/4000 [11:05<00:34,  5.71it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3641 / 3802:  95%|█████████▌| 3802/4000 [11:05<00:34,  5.71it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3642 / 3803:  95%|█████████▌| 3803/4000 [11:05<00:34,  5.71it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3642 / 3803:  95%|█████████▌| 3804/4000 [11:05<00:34,  5.71it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3643 / 3804:  95%|█████████▌| 3804/4000 [11:05<00:34,  5.71it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3644 / 3805:  95%|█████████▌| 3805/4000 [11:05<00:34,  5.72it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3645 / 3806:  95%|█████████▌| 3806/4000 [11:05<00:33,  5.72it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3645 / 3806:  95%|█████████▌| 3807/4000 [11:05<00:33,  5.72it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3646 / 3807:  95%|█████████▌| 3807/400


[Succeeded / Failed / Skipped / Total] 157 / 4 / 3740 / 3901:  98%|█████████▊| 3901/4000 [11:12<00:17,  5.80it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3741 / 3902:  98%|█████████▊| 3902/4000 [11:12<00:16,  5.80it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3741 / 3902:  98%|█████████▊| 3903/4000 [11:12<00:16,  5.80it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3742 / 3903:  98%|█████████▊| 3903/4000 [11:12<00:16,  5.80it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3743 / 3904:  98%|█████████▊| 3904/4000 [11:12<00:16,  5.80it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3744 / 3905:  98%|█████████▊| 3905/4000 [11:12<00:16,  5.80it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3744 / 3905:  98%|█████████▊| 3906/4000 [11:12<00:16,  5.80it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3745 / 3906:  98%|█████████▊| 3906/4000 [11:12<00:16,  5.80it/s]
[Succeeded / Failed / Skipped / Total] 157 / 4 / 3746 / 3907:  98%|█████████▊| 3907/400





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 161    |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 3835   |
| Original accuracy:            | 4.12%  |
| Accuracy under attack:        | 0.1%   |
| Attack success rate:          | 97.58% |
| Average perturbed word %:     | 10.79% |
| Average num. words per input: | 20.94  |
| Avg num queries:              | 73.3   |
+-------------------------------+--------+


 ...]

In [69]:
print(dataset[1:2])
print((train_data[0].guid))

[(OrderedDict([('sentence', 'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .')]), 1)]
train-1
